# Trabajo final

In [3]:
import pandas as pd

In [5]:
#Cargar las BD

anemia="C:/Users/spdie/OneDrive/Desktop/REC44_2023.csv"
anemiaBD=pd.read_csv(anemia, encoding = "latin-1")
anemiaBD.columns= anemiaBD.columns.str.lower()

hierro="C:/Users/spdie/OneDrive/Desktop/REC95_2023.csv"
hierroBD=pd.read_csv(hierro, encoding = "latin-1")
hierroBD.columns= hierroBD.columns.str.lower()

In [7]:
#Conservar data frames solo con las variables a utilizar

ane_small = anemiaBD[['caseid', 'hw57', 'hw13']]
anemia_small = ane_small.copy()
print(anemia_small.columns)

hie_small = hierroBD[['caseid','s465db_a', 's465db_b', 's465db_c', 's465db_d']]
hierro_small = hie_small.copy()
print(hierro_small.columns)

jun_small = juntosBD[['caseid', 's484']]
juntos_small = jun_small.copy()
print(juntos_small.columns)

ubi_small = ubicacionBD[['caseid', 'v024', 'v025']]
ubicacion_small = ubi_small.copy()
print(ubicacion_small.columns)

Index(['caseid', 'hw57', 'hw13'], dtype='object')
Index(['caseid', 's465db_a', 's465db_b', 's465db_c', 's465db_d'], dtype='object')
Index(['caseid', 's484'], dtype='object')
Index(['caseid', 'v024', 'v025'], dtype='object')


In [9]:
#Para este ejercicio solo conservaremos las observaciones en los que los niños tuvieron medición de anemia
anemia_filter = anemia_small.query('hw13 == 0')

#Verificar que en la variable solo haya 0 como observación, pues significa que fue medido
verificar = anemia_filter['hw13'].unique()
print(verificar)

[0]


In [113]:
#unir las BD. 

bd_merge = pd.merge(anemia_filter, hierro_small, on='caseid', how='inner')

print(bd_merge.shape)
print(bd_merge.columns)

(24849, 7)
Index(['caseid', 'hw57', 'hw13', 's465db_a', 's465db_b', 's465db_c',
       's465db_d'],
      dtype='object')


In [115]:
#Convertir a númericos los valores que utilizaré posteriormente
bd_merge['s465db_a'] = pd.to_numeric(bd_merge['s465db_a'], errors='coerce')
bd_merge['s465db_b'] = pd.to_numeric(bd_merge['s465db_b'], errors='coerce')
bd_merge['s465db_c'] = pd.to_numeric(bd_merge['s465db_c'], errors='coerce')
bd_merge['s465db_d'] = pd.to_numeric(bd_merge['s465db_d'], errors='coerce')

In [117]:
#eliminar datos pérdidos
bd_merge= bd_merge.dropna(subset=['s465db_a','s465db_b','s465db_c','s465db_d'])
print(bd_merge.shape)

(23379, 7)


In [119]:
#Crear nuevas variables dicotómicas, para saber si recibe o no el hierro en esa presentación por parte del MINSA
bd_merge[['h_jarabe','h_gotas','h_polvo','h_otro']] = bd_merge[['s465db_a','s465db_b','s465db_c','s465db_d']].replace({2: 0, 8: 0})

In [121]:
print(bd_merge.columns)

Index(['caseid', 'hw57', 'hw13', 's465db_a', 's465db_b', 's465db_c',
       's465db_d', 'h_jarabe', 'h_gotas', 'h_polvo', 'h_otro'],
      dtype='object')


In [123]:
#Quiero tener una variable que muestre en cuantos tipos de presentaciones reciben el hierro por parte del MINSA
bd_merge["pres_hierro"] = bd_merge['h_jarabe']+bd_merge['h_gotas']+bd_merge['h_polvo']+bd_merge['h_otro']

Quiero saber el porcentaje que no recibe ninguna presentación de hierro por parte del MINSA

In [126]:
def porcentaje_nr(df,variable):
    no_hierro =df[df[variable]==0]
    porcentaje = (len(no_hierro)/len(df))*100
    return "El {} no recibe ninguna presentación de hierro por parte del MINSA".format(porcentaje)

sin_hierro= porcentaje_nr(bd_merge,"pres_hierro")
print(sin_hierro)

El 42.268702681894005 no recibe ninguna presentación de hierro por parte del MINSA


Quiero un gráfico que muestre el porcentaje de aquellos con anemia en nivel moderado según la cantidad de presentaciones de hierro que reciben por parte del MINSA

In [69]:
import altair as alt

In [71]:
!pip install altair

In [75]:
#Variable nivel de anemia

#Nueva variable con valores
bd_merge ['anemia_cat'] = bd_merge ['hw57'].replace({"1": 'grave', "2": 'moderado', "3": 'leve', "4": 'sin anemia'})
print(bd_merge[['anemia_cat' , 'hw57']])

       anemia_cat hw57
0      sin anemia    4
1            leve    3
2      sin anemia    4
3      sin anemia    4
4      sin anemia    4
...           ...  ...
24844  sin anemia    4
24845        leve    3
24846  sin anemia    4
24847           9    9
24848  sin anemia    4

[23379 rows x 2 columns]


In [99]:
#Tener una sub data solo con los que tienen anemia moderada
bd_merge_sub= bd_merge.query("anemia_cat == 'moderado'")
print(bd_merge_sub.shape)

(2390, 16)


In [107]:
#Calcular los porcentajes a mostrar en el gráfico

contar_cant = bd_merge_sub['pres_hierro'].value_counts().reset_index()
contar_cant.columns = ['pres_hierro', 'cantidad']

contar_cant["porcentaje"] = contar_cant['cantidad'] / contar_cant['cantidad'].sum() * 100
contar_cant

,pres_hierro,cantidad,porcentaje
0,1.0,1087,45.481172
1,0.0,760,31.799163
2,2.0,463,19.372385
3,3.0,79,3.305439
4,4.0,1,0.041841


In [129]:
# Crear el gráfico

# Crear el gráfico de barras
chart = alt.Chart(contar_cant).mark_bar().encode(
    x='pres_hierro:O',  
    y='porcentaje:Q',  
    color='pres_hierro:N',  
    tooltip=['pres_hierro', 'porcentaje']).properties(title='Cantidad de presentaciones de hierro que reciben quienes tienen anemia moderada por parte del MINSA')

chart.show()

alt.Chart(...)